# For N1 590 GeV ; gl 600 GeV ; 100k events ; using CKKW

In [1]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-590-600/Events/run_100k

home_dir = "root_files/N590_G600/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Weight")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.113E+02  # pb for gluino at 600 GeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(6).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)

%reset -f


Welcome to JupyROOT 6.26/06

Initial number of events: 1582000.00

Number of events passed MET > 200 GeV cut: 397265.10

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 396590.97

Number of events passed number of jets <= 4 cut: 336623.74

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 269362.59

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 233841.09

For SR MET > 200 GeV: S_Exp: 233841.09, S_Obs: 102274, and ratio: 2.29

For SR MET > 250 GeV: S_Exp: 185171.01, S_Obs: 41158, and ratio: 4.50

For SR MET > 300 GeV: S_Exp: 140390.80, S_Obs: 20893, and ratio: 6.72

For SR MET > 350 GeV: S_Exp: 106721.88, S_Obs: 11937, and ratio: 8.94

For SR MET > 400 GeV: S_Exp: 81767.95, S_Obs: 7214, and ratio: 11.33

For SR MET > 500 GeV: S_Exp: 48460.25, S_Obs: 2918, and ratio: 16.61

For SR MET > 600 GeV: S_Exp: 29448.24, S_Obs: 1391, and ratio: 21.17

For SR MET > 700 GeV: S_Exp: 18690.78, S_Obs: 574, and ratio: 32.56

For SR MET > 800 

# For N1 965 GeV ; gl 975 GeV ; 100k events ; using CKKW

In [2]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-965-975/Events/run_100k

home_dir = "root_files/N965_G975/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Weight")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.462  # pb for gluino at 975 GeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(6).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)

%reset -f



Initial number of events: 64680.00

Number of events passed MET > 200 GeV cut: 16261.99

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 16227.51

Number of events passed number of jets <= 4 cut: 13812.97

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 11075.79

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 9629.54

For SR MET > 200 GeV: S_Exp: 9629.54, S_Obs: 102274, and ratio: 0.09

For SR MET > 250 GeV: S_Exp: 7594.71, S_Obs: 41158, and ratio: 0.18

For SR MET > 300 GeV: S_Exp: 5801.40, S_Obs: 20893, and ratio: 0.28

For SR MET > 350 GeV: S_Exp: 4395.24, S_Obs: 11937, and ratio: 0.37

For SR MET > 400 GeV: S_Exp: 3364.46, S_Obs: 7214, and ratio: 0.47

For SR MET > 500 GeV: S_Exp: 1971.09, S_Obs: 2918, and ratio: 0.68

For SR MET > 600 GeV: S_Exp: 1191.93, S_Obs: 1391, and ratio: 0.86

For SR MET > 700 GeV: S_Exp: 775.81, S_Obs: 574, and ratio: 1.35

For SR MET > 800 GeV: S_Exp: 475.77, S_Obs: 298, and ratio: 1.60

For 

# For N1 850 GeV ; gl 1000 GeV ; 100k events ; using CKKW

In [3]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-850-1000/Events/run_100k

home_dir = "root_files/N850_G1000/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Weight")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.385  # pb for gluino at 1 TeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(6).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)

%reset -f



Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13528.20

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13499.86

Number of events passed number of jets <= 4 cut: 11459.50

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9174.01

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7964.87

For SR MET > 200 GeV: S_Exp: 7964.87, S_Obs: 102274, and ratio: 0.08

For SR MET > 250 GeV: S_Exp: 6302.44, S_Obs: 41158, and ratio: 0.15

For SR MET > 300 GeV: S_Exp: 4774.13, S_Obs: 20893, and ratio: 0.23

For SR MET > 350 GeV: S_Exp: 3622.55, S_Obs: 11937, and ratio: 0.30

For SR MET > 400 GeV: S_Exp: 2777.26, S_Obs: 7214, and ratio: 0.38

For SR MET > 500 GeV: S_Exp: 1645.79, S_Obs: 2918, and ratio: 0.56

For SR MET > 600 GeV: S_Exp: 1007.38, S_Obs: 1391, and ratio: 0.72

For SR MET > 700 GeV: S_Exp: 639.91, S_Obs: 574, and ratio: 1.11

For SR MET > 800 GeV: S_Exp: 400.34, S_Obs: 298, and ratio: 1.34

For S

# For N1 900 GeV ; gl 1000 GeV ; 100k events ; using CKKW

In [4]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-900-1000/Events/run_100k

home_dir = "root_files/N900_G1000/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Weight")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.385  # pb for gluino at 1 TeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(6).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)

%reset -f



Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13585.72

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13556.38

Number of events passed number of jets <= 4 cut: 11520.81

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9255.22

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8020.01

For SR MET > 200 GeV: S_Exp: 8020.01, S_Obs: 102274, and ratio: 0.08

For SR MET > 250 GeV: S_Exp: 6310.31, S_Obs: 41158, and ratio: 0.15

For SR MET > 300 GeV: S_Exp: 4815.42, S_Obs: 20893, and ratio: 0.23

For SR MET > 350 GeV: S_Exp: 3650.73, S_Obs: 11937, and ratio: 0.31

For SR MET > 400 GeV: S_Exp: 2809.05, S_Obs: 7214, and ratio: 0.39

For SR MET > 500 GeV: S_Exp: 1668.44, S_Obs: 2918, and ratio: 0.57

For SR MET > 600 GeV: S_Exp: 1004.07, S_Obs: 1391, and ratio: 0.72

For SR MET > 700 GeV: S_Exp: 650.84, S_Obs: 574, and ratio: 1.13

For SR MET > 800 GeV: S_Exp: 399.13, S_Obs: 298, and ratio: 1.34

For S

# For N1 950 GeV ; gl 1000 GeV ; 100k events ; using CKKW

In [5]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-950-1000/Events/run_100k

home_dir = "root_files/N950_G1000/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Weight")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.385  # pb for gluino at 1 TeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(6).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)

%reset -f



Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13547.85

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13519.67

Number of events passed number of jets <= 4 cut: 11470.67

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9213.26

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8004.70

For SR MET > 200 GeV: S_Exp: 8004.70, S_Obs: 102274, and ratio: 0.08

For SR MET > 250 GeV: S_Exp: 6315.89, S_Obs: 41158, and ratio: 0.15

For SR MET > 300 GeV: S_Exp: 4811.75, S_Obs: 20893, and ratio: 0.23

For SR MET > 350 GeV: S_Exp: 3634.22, S_Obs: 11937, and ratio: 0.30

For SR MET > 400 GeV: S_Exp: 2769.77, S_Obs: 7214, and ratio: 0.38

For SR MET > 500 GeV: S_Exp: 1631.25, S_Obs: 2918, and ratio: 0.56

For SR MET > 600 GeV: S_Exp: 993.93, S_Obs: 1391, and ratio: 0.71

For SR MET > 700 GeV: S_Exp: 635.81, S_Obs: 574, and ratio: 1.11

For SR MET > 800 GeV: S_Exp: 392.77, S_Obs: 298, and ratio: 1.32

For SR

# For N1 990 GeV ; gl 1000 GeV ; 100k events ; using CKKW

In [6]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-990-1000/Events/run_100k

home_dir = "root_files/N990_G1000/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Weight")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.385  # pb for gluino at 1 TeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(6).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)

%reset -f



Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13549.63

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13524.41

Number of events passed number of jets <= 4 cut: 11443.87

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9223.06

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7988.20

For SR MET > 200 GeV: S_Exp: 7988.20, S_Obs: 102274, and ratio: 0.08

For SR MET > 250 GeV: S_Exp: 6318.24, S_Obs: 41158, and ratio: 0.15

For SR MET > 300 GeV: S_Exp: 4827.87, S_Obs: 20893, and ratio: 0.23

For SR MET > 350 GeV: S_Exp: 3643.84, S_Obs: 11937, and ratio: 0.31

For SR MET > 400 GeV: S_Exp: 2776.32, S_Obs: 7214, and ratio: 0.38

For SR MET > 500 GeV: S_Exp: 1652.42, S_Obs: 2918, and ratio: 0.57

For SR MET > 600 GeV: S_Exp: 1013.94, S_Obs: 1391, and ratio: 0.73

For SR MET > 700 GeV: S_Exp: 641.93, S_Obs: 574, and ratio: 1.12

For SR MET > 800 GeV: S_Exp: 401.61, S_Obs: 298, and ratio: 1.35

For S

# For N1 995 GeV ; gl 1000 GeV ; 100k events ; using CKKW

In [7]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-995-1000/Events/run_100k

home_dir = "root_files/N995_G1000/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Weight")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.385  # pb for gluino at 1 TeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(6).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)

%reset -f



Initial number of events: 53900.00

Number of events passed MET > 200 GeV cut: 13528.92

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13504.86

Number of events passed number of jets <= 4 cut: 11415.72

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9186.02

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7994.23

For SR MET > 200 GeV: S_Exp: 7994.23, S_Obs: 102274, and ratio: 0.08

For SR MET > 250 GeV: S_Exp: 6300.65, S_Obs: 41158, and ratio: 0.15

For SR MET > 300 GeV: S_Exp: 4796.62, S_Obs: 20893, and ratio: 0.23

For SR MET > 350 GeV: S_Exp: 3618.33, S_Obs: 11937, and ratio: 0.30

For SR MET > 400 GeV: S_Exp: 2763.36, S_Obs: 7214, and ratio: 0.38

For SR MET > 500 GeV: S_Exp: 1656.42, S_Obs: 2918, and ratio: 0.57

For SR MET > 600 GeV: S_Exp: 1010.00, S_Obs: 1391, and ratio: 0.73

For SR MET > 700 GeV: S_Exp: 654.97, S_Obs: 574, and ratio: 1.14

For SR MET > 800 GeV: S_Exp: 400.76, S_Obs: 298, and ratio: 1.34

For S

# For N1 1015 GeV ; gl 1025 GeV ; 100k events ; using CKKW

In [8]:
import ROOT
import math

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

# Set the path to the ROOT file
# /data/analysis/iduminic/Hammad/MSSM_LO_gogo_TESTCKKW_V2-1015-1025/Events/run_100k

home_dir = "root_files/N1015_G1025/"
inputFile = home_dir + "tag_1_delphes_events.root"

# Create chain of root trees
chain = ROOT.TChain("Delphes")
chain.Add(inputFile)

# Create object of class ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(chain)
numberOfEntries = treeReader.GetEntries()

# Get pointer to the branch containing Particle information
branchParticle = treeReader.UseBranch("Particle")
branchJet = treeReader.UseBranch("Jet")
branchMET = treeReader.UseBranch("MissingET")
branchEvent = treeReader.UseBranch("Weight")

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)
def calculate_delta_phi_jet_met(jet_phi, met_phi): return delta_phi(jet_phi, met_phi)

# Events Counter
initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0

#### Initial Cuts 
pt_cut = 30.0
max_eta = 2.8  

# Constants for scaling
luminosity = 140.0 * 1000  # pb
cross_section = 0.322  # pb for gluino at 1025 GeV https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
# W = σ * L * Wgen / ΣWgen. https://ipnp.cz/scheirich/?page_id=292

# Dictionary to store MET cut values and corresponding event counts
met_cuts = { 200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

sum_event_weights = 0
for i in range(numberOfEntries):
    treeReader.ReadEntry(i)
    sum_event_weights += branchEvent.At(6).Weight

# Loop over all events
for entry in range(numberOfEntries):
    # Load selected branches with data from specified event
    treeReader.ReadEntry(entry)
    
    num_jets_meeting_criteria = leading_jet_pt =  leading_jet_eta = 0

    initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
    
    if branchMET.At(0).MET > 200:
        events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
    
        # Loop over all jets
        for jet_index in range(branchJet.GetEntries()):
            jet = branchJet.At(jet_index)
        
            # Jet Pt and Eta Cuts
            if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                num_jets_meeting_criteria += 1
            
            # Save Leading Jet Pt and Eta
            if jet.PT > leading_jet_pt:
                leading_jet_pt = jet.PT
                leading_jet_eta = jet.Eta

        # Count events passing pt, eta cut
        if num_jets_meeting_criteria > 0:
            events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
        
        # To set limit over jets
        if num_jets_meeting_criteria <= 4:
            events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            #print("Number of jets meeting criteria in event {}: {}".format(entry, num_jets_meeting_criteria)) #to check number of jets in each event
            
            #Delta_Phi cut implementation
            delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
            delta_phis = []
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)
                delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                #delta_phi_jet_met = ROOT.TMath.Abs(ROOT.Math.VectorUtil.DeltaPhi(jet.Phi, branchMET.At(0).Phi))
                delta_phis.append(delta_phi_jet_met)
            
            # Check how many jets pass the delta_phi cut
            if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            
                # Leading Jets Pt and Eta cuts
                if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                    events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                
                    # Loop over MET cut values
                    for met_cut in met_cuts:
                        if branchMET.At(0).MET > met_cut:
                            met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights

print(f"\nInitial number of events: {initial_events:.2f}")
print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f}")
print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f}")
print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f}")
print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f}")
print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f}")

paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                  600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

ratios = {}  # Dictionary to store the ratios

# Calculate the ratio for each MET cut value
for met_cut in met_cuts:
    if met_cut in paper_met_cuts:
        ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
        ratios[met_cut] = ratio
        print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f}")
        
max_ratio_met_cut = max(ratios, key=ratios.get)

gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point)

%reset -f



Initial number of events: 45080.00

Number of events passed MET > 200 GeV cut: 11300.30

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 11277.46

Number of events passed number of jets <= 4 cut: 9564.76

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 7647.05

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 6623.78

For SR MET > 200 GeV: S_Exp: 6623.78, S_Obs: 102274, and ratio: 0.06

For SR MET > 250 GeV: S_Exp: 5181.02, S_Obs: 41158, and ratio: 0.13

For SR MET > 300 GeV: S_Exp: 3974.80, S_Obs: 20893, and ratio: 0.19

For SR MET > 350 GeV: S_Exp: 3021.88, S_Obs: 11937, and ratio: 0.25

For SR MET > 400 GeV: S_Exp: 2303.71, S_Obs: 7214, and ratio: 0.32

For SR MET > 500 GeV: S_Exp: 1371.91, S_Obs: 2918, and ratio: 0.47

For SR MET > 600 GeV: S_Exp: 832.61, S_Obs: 1391, and ratio: 0.60

For SR MET > 700 GeV: S_Exp: 513.93, S_Obs: 574, and ratio: 0.90

For SR MET > 800 GeV: S_Exp: 324.45, S_Obs: 298, and ratio: 1.09

For SR 